In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
dataset = pd.read_csv('project3_dataset4.txt', sep="\t", header=None)
nrows = len(dataset)
ncol = len(dataset.columns)
print(dataset)

           0     1       2       3  4
0      sunny   hot    high    weak  0
1      sunny   hot    high  strong  0
2   overcast   hot    high    weak  1
3       rain  mild    high    weak  1
4       rain  cool  normal    weak  1
5       rain  cool  normal  strong  0
6   overcast  cool  normal  strong  1
7      sunny  mild    high    weak  0
8      sunny  cool  normal    weak  1
9       rain  mild  normal    weak  1
10     sunny  mild  normal  strong  1
11  overcast  mild    high  strong  1
12  overcast   hot  normal    weak  1
13      rain  mild    high  strong  0


In [3]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [4]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [5]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical         
        else:
            potential_splits[column_index] = unique_values
    
    return potential_splits


In [6]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [7]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [8]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [9]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [10]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [11]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global  FEATURE_TYPES
        #COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        #feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(split_column, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(split_column, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [12]:
tree = decision_tree_algorithm(dataset)#, max_depth=5)
pprint(tree)

{'0 = overcast': [1,
                  {'2 = normal': [{'3 = weak': [1, {'1 = mild': [1, 0]}]},
                                  {'0 = sunny': [0, {'3 = weak': [1, 0]}]}]}]}


In [13]:
{'0 <= overcast': [1, {'2 <= normal': [{'3 <= weak': [1, {'1 <= mild': [1, 0]}]},
                                       {'0 <= sunny': [0, {'3 <= weak': [1, 0]}]}]}]}

{'0 <= overcast': [1,
  {'2 <= normal': [{'3 <= weak': [1, {'1 <= mild': [1, 0]}]},
    {'0 <= sunny': [0, {'3 <= weak': [1, 0]}]}]}]}

In [14]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[int(feature_name)] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

example = np.array(dataset)
print(example)
classes1 =[]
for i in example:
    classes1.append(classify_example(i,tree))  
print(classes1)

[['sunny' 'hot' 'high' 'weak' 0]
 ['sunny' 'hot' 'high' 'strong' 0]
 ['overcast' 'hot' 'high' 'weak' 1]
 ['rain' 'mild' 'high' 'weak' 1]
 ['rain' 'cool' 'normal' 'weak' 1]
 ['rain' 'cool' 'normal' 'strong' 0]
 ['overcast' 'cool' 'normal' 'strong' 1]
 ['sunny' 'mild' 'high' 'weak' 0]
 ['sunny' 'cool' 'normal' 'weak' 1]
 ['rain' 'mild' 'normal' 'weak' 1]
 ['sunny' 'mild' 'normal' 'strong' 1]
 ['overcast' 'mild' 'high' 'strong' 1]
 ['overcast' 'hot' 'normal' 'weak' 1]
 ['rain' 'mild' 'high' 'strong' 0]]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
def Get_Acc_new(Y,target):
    Y =np.array(Y)
    target=np.array(target)
    accuracy = 0.0
    counter = 0
    for i in range(len(Y)):
            if Y[i] == target[i]:
                counter= counter+1
    accuracy = (float((counter)*100)/float(len(Y)))
    return accuracy
Get_Acc_new(classes1,example[:,ncol-1])